# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Dimension-reduction" data-toc-modified-id="Dimension-reduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dimension reduction</a></div>

In [2]:
#library(ggbiplot)
library(tidyverse, quiet=T)

library(caret, quiet=T)

dat0 <- read.csv("../datasets/Ultrasound/training_ultrasound.csv")

# Use Hadlok equation to estimate fetal weight from the 4 key ultrasound measurements
dat.raw = dat0 %>%
    mutate(
        LOG10.FWT.GM = 1.3596 + 0.0064*HCIRCM + 0.0424*ABCIRCM + 0.174*FEMURCM + 0.00061*BPDCM*ABCIRCM - 0.00386*ABCIRCM*FEMURCM,
        WTKG.estimate = ifelse(AGEDAYS<1 ,(10^LOG10.FWT.GM)/1000 ,WTKG) ,
        Study = paste('Study', STUDYID)
        )

dat = dat.raw %>% group_by(SUBJID) %>%
    mutate(tot.measurements = n()) %>%
    filter(tot.measurements >1)

# Dimension reduction

We use PCA to reduce the four physical variables to one single dimension

In [47]:
# clean data, remove samples with only one observation
dat.covars = dat %>% ungroup %>%
    select(ABCIRCM, HCIRCM, BPDCM, FEMURCM)

dat.preproc = dat.covars  %>%
    mutate_each(funs(as.numeric(.))) %>%
    preProcess(., method=c("center", "scale", "knnImpute", "pca"))

In [61]:
dat.prcomp = dat.preproc$data %>% prcomp()


dat.prcomp$x %>% nrow
#dat.preproc$data %>% ungroup %>% mutate(
#    PC1=
#        ABCIRCM*dat.preproc$rotation[1] +
#        HCIRCM *dat.preproc$rotation[2] +
#        BPDCM  *dat.preproc$rotation[3] +
#        FEMURCM*dat.preproc$rotation[4]
#    ) %>%
#    nrow

dat.covars %>% filter(complete.cases(.)) %>% nrow

[1] 7940

[1] 7940

In [21]:
dat.prcomp$x %>% nrow
dat.covars %>% nrow
dat.prcomp$rotation

[1] 7940

[1] 17336

,PC1,PC2,PC3,PC4
ABCIRCM,-0.5377268,-0.7129458,0.44874284,-0.03446891
HCIRCM,-0.3759303,0.2759221,0.05571315,0.88285870
BPDCM,-0.5303786,0.6356879,0.34014992,-0.44597912
FEMURCM,-0.5368658,-0.1071268,-0.82451437,-0.14309101
